# ᗜˬᗜ Detector de Fumos con YOLOv8 de vuhn

In [ ]:
# Instalamos ultralytics y tqdm para usar YOLOv8 y mostrar progreso
!pip install ultralytics tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
# Subimos el archivo data.zip exportado desde Label Studio con las imágenes y etiquetas
# Luego lo descomprimimos en la carpeta 'raw_dataset'
from zipfile import ZipFile
with ZipFile('data.zip', 'r') as z:
    z.extractall('raw_dataset')

In [ ]:
# Dividimos el dataset en entrenamiento (train) y validación (val), con una proporción de 80/20. Las imágenes y etiquetas se mueven a las carpetas necesarias.
import os, shutil, random
from tqdm import tqdm

src_dir     = 'raw_dataset'
dest_folder = 'fumo_dataset'
split_ratio = 0.8 #Puedes cambiar este valor para variar la proporción entre imágenes de entrenamiento e imágenes de validación.
seed_val    = 42

train_imgs = os.path.join(dest_folder, 'train', 'images')
train_lbls = os.path.join(dest_folder, 'train', 'labels')
val_imgs   = os.path.join(dest_folder, 'val',   'images')
val_lbls   = os.path.join(dest_folder, 'val',   'labels')

for path in [train_imgs, train_lbls, val_imgs, val_lbls]:
    os.makedirs(path, exist_ok=True)

all_files = [f for f in os.listdir(os.path.join(src_dir,'images')) if f.lower().endswith(('.png','.jpg','.jpeg'))]
random.seed(seed_val)
random.shuffle(all_files)

split_index = int(len(all_files) * split_ratio)
train_files = all_files[:split_index]
val_files   = all_files[split_index:]

for subset, file_list in zip(['train','val'], [train_files, val_files]):
    for fname in tqdm(file_list, desc=f'Copiando {subset}'):

        shutil.copy(
            os.path.join(src_dir,'images', fname),
            os.path.join(dest_folder, subset, 'images', fname)
        )

        label_name = os.path.splitext(fname)[0] + '.txt'
        shutil.copy(
            os.path.join(src_dir,'labels', label_name),
            os.path.join(dest_folder, subset, 'labels', label_name)
        )

Copiando val: 100%|██████████| 71/71 [00:00<00:00, 4619.64it/s]


In [ ]:
# Creamos el archivo de configuración YAML con los datos del dataset y las clases.
yaml_content = """\
# fumo_config.yaml
path: /content/fumo_dataset
train: train/images
val:   val/images

nc: 1
names: ['Fumo']
"""
with open('fumo_config.yaml', 'w') as f:
    f.write(yaml_content)

Creado /content/fumo_config.yaml con estas rutas:
# fumo_config.yaml
path: /content/fumo_dataset
train: train/images
val:   val/images

nc: 1
names: ['Fumo']



In [ ]:
# Entrenamos el modelo YOLOv8 con nuestro dataset personalizado
# Guardamos los resultados en la carpeta 'models/colab_fumo_v1'
from ultralytics import YOLO

model = YOLO('yolov8s.pt')
model.train(
    data='fumo_config.yaml',
    epochs=50,
    imgsz=640,
    batch=12,
    project='models',
    name='colab_fumo_v1',
    exist_ok=True
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 21.5M/21.5M [00:00<00:00, 472MB/s]


Ultralytics 8.3.128 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=12, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=fumo_config.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=colab_fumo_v1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True

100%|██████████| 755k/755k [00:00<00:00, 116MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

...

                   all         71        113      0.938      0.982       0.99      0.794



50 epochs completed in 0.098 hours.
Optimizer stripped from models/colab_fumo_v1/weights/last.pt, 22.5MB
Optimizer stripped from models/colab_fumo_v1/weights/best.pt, 22.5MB

Validating models/colab_fumo_v1/weights/best.pt...
Ultralytics 8.3.128 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.06it/s]


                   all         71        113      0.937      0.982      0.989      0.795
Speed: 0.4ms preprocess, 5.5ms inference, 0.0ms loss, 3.5ms postprocess per image
Results saved to models/colab_fumo_v1


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7eca500909d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
# Descargamos el modelo entrenado (best.pt) para usarlo localmente.
from google.colab import files
files.download('models/colab_fumo_v1/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Código para detección en tiempo real con webcam usando OpenCV y el modelo entrenado
# Esto ejecuta un archivo .py en la carpeta models
from ultralytics import YOLO
import cv2

model = YOLO('models/best_fumo_v1.pt')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(source=frame, show=True, conf=0.4)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()